### Imports

In [ ]:
from pprint import pprint

import numpy as np

In [ ]:
from rayuela.base.semiring import Boolean, Real, Tropical
from rayuela.base.symbol import Sym, ε
from rayuela.fsa.fsa import FSA
from rayuela.fsa.state import State
from rayuela.fsa.pathsum import Pathsum, Strategy

## Example 1

In [ ]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(3), Sym('a'), State(6))
fsa.add_arc(State(3), Sym('b'), State(1))

fsa.add_arc(State(6), Sym('c'), State(5))

fsa.add_arc(State(1), Sym('c'), State(5))
fsa.add_arc(State(1), Sym('b'), State(4))

fsa.add_arc(State(4), Sym('a'), State(2))

fsa.add_arc(State(5), Sym('a'), State(2))

fsa.set_I(State(3))
fsa.set_F(State(2))

fsa

In [ ]:
list(fsa.toposort())

## Example 2

Backward with all the intermediate β values

In [ ]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(1))

fsa.add_arc(State(3), Sym('a'), State(6), Real(1))
fsa.add_arc(State(3), Sym('b'), State(1), Real(7))

fsa.add_arc(State(6), Sym('c'), State(5), Real(4))

fsa.add_arc(State(1), Sym('c'), State(5), Real(3))
fsa.add_arc(State(1), Sym('b'), State(4), Real(1))

fsa.add_arc(State(4), Sym('a'), State(2), Real(5))

fsa.add_arc(State(5), Sym('a'), State(2), Real(3))
fsa.add_arc(State(5), Sym('a'), State(7), Real(8))

fsa.add_arc(State(6), Sym('a'), State(7), Real(7))

fsa.add_arc(State(7), Sym('a'), State(8), Real(2))

fsa.set_I(State(3), Real(3))
fsa.set_F(State(2), Real(4))
fsa.set_F(State(7), Real(2))

fsa

In [ ]:
pprint(Pathsum(fsa).backward(strategy=Strategy.VITERBI))

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.VITERBI))

## Example 3

Matrix closure by fixed point

We can compute $\left( I - M\right)^{-1}$ using the fixed point algorithm ran until numerical convergence.

In [ ]:
D = 5

M = np.random.rand(D, D) / D  # ensure entries are smaller than 1 / D
M

In [ ]:
R = np.zeros_like(M)
for i in range(200):
    R = (np.eye(D, D) + R) @ M
np.eye(D, D) + R

In [ ]:
np.linalg.inv(np.eye(D, D) - M)

Matrix power in logarithmic number of multiplications

In [ ]:
M = np.random.rand(D, D)

In [ ]:
m = 13
# 13 = 8 + 4 + 1 = 2^3 + 2^2 + 2^0

In [ ]:
Ms = [M]
for r in range(1, 4):
    Ms.append(Ms[-1] @ Ms[-1])

In [ ]:
Ms[0] @ Ms[2] @ Ms[3]

In [ ]:
np.linalg.matrix_power(M, 13)

### Graph closure by WFK

The closure of the graph answers the question whether there is a path between pairs of nodes for each possible pair

#### Acyclic

In [ ]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))
fsa.add_arc(State(5), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

#### A self-loop

In [ ]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))
fsa.add_arc(State(5), Sym('a'), State(5))
fsa.add_arc(State(5), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

#### A cycle

In [ ]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))

fsa.add_arc(State(7), Sym('a'), State(5))

fsa.add_arc(State(8), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

The entire matrix also shows which nodes are definitely *not* in the same strongly connected component

### Pathsum by Lehmann

#### Acyclic

In [ ]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))
fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

#### A self-loop

In [ ]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))
fsa.add_arc(State(5), Sym('a'), State(5), Real(0.9))
fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

#### A cycle

In [ ]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))

fsa.add_arc(State(7), Sym('a'), State(5), Real(0.3))

fsa.add_arc(State(8), Sym('a'), State(7), Real(0.3))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

### Shortest paths by Lehmann (Floyd-Warshall)

#### Acyclic

In [ ]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))
fsa.add_arc(State(5), Sym('a'), State(7), Tropical(8))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.VITERBI))

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

In [ ]:
pprint(Pathsum(fsa).lehmann())

#### A self-loop

In [ ]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))
fsa.add_arc(State(5), Sym('a'), State(5), Tropical(9))
fsa.add_arc(State(5), Sym('a'), State(7), Tropical(8))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

We see that since the Tropical semiring is 0-closed, the self loop does not affect the pathsum

#### A Cycle

In [ ]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))

fsa.add_arc(State(7), Sym('a'), State(5), Tropical(3))

fsa.add_arc(State(8), Sym('a'), State(7), Tropical(3))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

In [ ]:
pprint(Pathsum(fsa).lehmann())

In [ ]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

Again, the cycle does not affect the pathsum

## Example 4

Connected components by hand

In [ ]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(5), Real(0.1))

fsa.add_arc(State(3), Sym('a'), State(6), Real(0.1))
fsa.add_arc(State(3), Sym('b'), State(1), Real(0.7))

fsa.add_arc(State(1), Sym('c'), State(3), Real(0.3))
fsa.add_arc(State(1), Sym('c'), State(5), Real(0.3))
fsa.add_arc(State(1), Sym('b'), State(4), Real(0.1))

fsa.add_arc(State(4), Sym('a'), State(2), Real(0.5))

fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.add_arc(State(6), Sym('a'), State(7), Real(0.7))
fsa.add_arc(State(6), Sym('a'), State(3), Real(0.1))

fsa.add_arc(State(7), Sym('a'), State(8), Real(0.2))

fsa.add_arc(State(8), Sym('a'), State(7), Real(0.2))
fsa.add_arc(State(8), Sym('a'), State(2), Real(0.2))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

In [ ]:
list(fsa.finish())